In [6]:
import os
import src.utils as utils
import src.Parser.verilog_parser as verilog_parser
import src.Parser.bench_parser as bench_parser
import src.Parser.conv as conv
from src.Netlist.AST import AST,module
import src.Attacks.SATAttack.SATAttack as satattack

In [7]:
locked_filename="input_files/benchmark_bench/rnd/apex2_enc05.bench"
unlocked_filename="input_files/benchmark_bench/original/apex2.bench"



top="apex2_enc05"
locked = AST(file_path=f"./output_files/{top}org.json",rw='r',filename=f"{top}locked_test") #Run to read in AST Format
top="apex2"
unlocked = AST(file_path=f"./output_files/{top}org.json",rw='r',filename=f"{top}locked_test") #Run to read in AST Format



extract_modules_def took 0.0005779266357421875 seconds to complete its execution.
Reading LL file
	 Loading json file
	 Done Loading json file
	 Loading Top module in AST
	 Done Loading Top module apex2_enc05 in AST
	 Loading module data in AST
		 Loading module apex2_enc05 in AST
		 Done Loading module apex2_enc05 in AST
	 Done Loading module data in AST
	 Loading gate level data in AST
	 Done Loading gate level data in AST
Done Reading LL file
extract_modules_def took 0.0010027885437011719 seconds to complete its execution.
Reading LL file
	 Loading json file
	 Done Loading json file
	 Loading Top module in AST
	 Done Loading Top module apex2 in AST
	 Loading module data in AST
		 Loading module apex2 in AST
		 Done Loading module apex2 in AST
	 Done Loading module data in AST
	 Loading gate level data in AST
	 Done Loading gate level data in AST
Done Reading LL file


In [15]:
for i,j in unlocked.top_module.io["outputs"].items():
  if(j["bits"]==1):
    print(i,j["bits"]==1)
  break

po2 True


In [ ]:
self.modules[key].module_name = key
self.modules[key].org_code_verilog = self.extracted_modules[key]
self.modules[key].gate_level_verilog = self.modules_techmap[key]
self.modules[key].gates,self.modules[key].linkages,tmp = verilog_parser.gates_module_extraction(self.modules[key].gate_level_verilog,self.gate_mapping_vlib,self.gates_vlib,self.FF_vlib)
self.modules[key].FF_tech,self.modules[key].Clock_pins,self.modules[key].Reset_pins=tmp


# print(self.modules[key].linkages.keys())

inputs, input_ports = extract_io_v(self.modules[key].org_code_verilog)
outputs, output_ports = extract_io_v(self.modules[key].org_code_verilog, "output")
wire, _ = extract_io_v(self.modules[key].gate_level_verilog, "wire")
with open("tmp/tmp.v","w") as f:
    f.write(self.modules[key].gate_level_verilog)
wire={key:wire[key]  for key in get_difference_abs(wire.keys(),inputs.keys(),outputs.keys())}


for i in self.modules[key].Clock_pins:
    if(("1'" in i) or (i not in inputs.keys())):
        self.modules[key].Clock_pins.remove(i)
# print("N4944" in  wire.keys())
# print(get_diference(wire.keys(),outputs.keys()))
# print("HERE",wire["cpuregs[5]"])

# print("N1947" in outputs.keys())
# print(self.modules[key].gates)
self.modules[key].io = dict({"Clock_pins":self.modules[key].Clock_pins,"Reset_pins":self.modules[key].Reset_pins,'wires':wire,'inputs':inputs,'outputs':outputs,'input_ports':input_ports,'output_ports':output_ports})
self.modules[key].gen_graph()
self.top_module=self.modules[self.top_module_name]

In [9]:
module()

In [4]:
print("START SAT ATTACK")
satobj=satattack.SatAttack(file_type="obj",locked_obj=locked.top_module,unlocked_obj=unlocked.top_module)
# satobj=satattack.SatAttack(file_type="b",locked_filename=locked_filename, unlocked_filename=unlocked_filename)
print(satobj.run())
print(satobj.iterations)

START SAT ATTACK
1000001010011011111101001111001
21


In [ ]:
top="fsm_0_obf"
pathin="input_files/ASSURE_LOCKED/design1/design1_netlist.v"

# top="fsm"
# pathin="input_files/ASSURE_LOCKED/design1/oracle1_netlist.v"

# utils.clean_dir("./tmp")

# obj=AST(file_path=pathin,rw="w",flag="v",top=top,filename=f"{top}org",vlibpath="input_files/ASSURE_LOCKED/modulefiles.v",synth=False)#Run to Read in Verilog Design

locked = AST(file_path=f"./output_files/{top}org.json",rw='r',filename=f"{top}locked_test") #Run to read in AST Format

In [ ]:
gate_mapping_vlib,gates_vlib,FF_vlib=locked.gate_mapping_vlib,locked.gates_vlib,locked.FF_vlib

In [ ]:
locked.top_module.gates.keys(),locked.top_module.FF_tech.keys()

In [ ]:
# tech_map={i:i for i in list(locked.top_module.gates,locked.top_module.FF_tech)}

In [ ]:
# for gate, gate_list in gate_mapping_vlib.items():
#   print(gate, gate_list)

In [ ]:
# path="/mnt/d/alis_files/LAPTOP/alis_files/university_files/PROJECTS_2022-2023/FYP/linux/65GP_STDCELL_verilog/tcbn65gplus.v"
# path= "vlib/mycells.v"
path="input_files/warm-up_obfuscation/internal_lib.v"
# path="/mnt/d/alis_files/LAPTOP/alis_files/university_files/PROJECTS_2022-2023/FYP/linux/gnnre/Netlist_to_graph/Circuits_datasets/Interconnected-Modules/Train_add_mul_sub_64_bit_Syn_65nm.v"
with open(path,"r") as f:
  module_txt=f.read()
gate_mapping_vlib2,gates_vlib2,FF_vlib2=verilog_parser.extract_modules_def(module_txt)

In [ ]:
def match_FF(FF1,FF2):
  FF1_I=FF1["inputs"]
  FF2_I=FF2["inputs"].copy()
  if(len(FF1_I)!=len(FF2_I)):
    return False
  # print(FF1_I,FF2_I)
  count=0
  for i in FF1_I:
    for j in FF2_I:
      if(utils.det_FF_node(i)==utils.det_FF_node(j)):
        # print(i,j)
        count+=1
        # FF1_I.remove(i)
        FF2_I.remove(j)
        break
  
  return count==len(FF1_I)

def gen_FF_techmap(FF_target,FF_source):
  # tech_map={target:[source]}
  tech_map={}
  for j in FF_source:
    for i in FF_target:
      if(match_FF(FF_target[i],FF_source[j])):
        if(i not in tech_map):
          tech_map[j]=i
          break
        # else:
        #   tech_map[i].append(j)
  return tech_map


In [ ]:
# tech_map=gen_FF_techmap(FF_vlib2,FF_vlib)
# print(FF_to_txt_techmap(locked.top_module.FF_tech,FF_vlib2,tech_map))

In [ ]:
def gen_tech_map(gate_mapping_vlib,FF_vlib,gates_vlib,gate_mapping_vlib2,FF_vlib2,gates_vlib2):
  gate_mapping_vlib=gate_mapping_vlib.copy()
  gate_mapping_vlib.pop("FF")
  tech_map=gen_FF_techmap(FF_vlib2,FF_vlib)

  for i in gate_mapping_vlib:
    for org_gate in gate_mapping_vlib[i]:
      for target_gate in gate_mapping_vlib2[i]:
        if(len(gates_vlib[org_gate]['inputs'])==len(gates_vlib2[target_gate]['inputs'])):
          tech_map[org_gate]=target_gate
          break
  return tech_map



In [ ]:
gate_mapping_vlib.keys()

In [ ]:
tech_map=gen_tech_map(gate_mapping_vlib,FF_vlib,gates_vlib,gate_mapping_vlib2,FF_vlib2,gates_vlib2)

In [ ]:
locked.top_module.gates["NOT"].items()

In [ ]:
def gates_to_txt_techmap(gates_tech,gates_vlib_2,tech_map):
  txt=""
  for _,logic_gates in gates_tech.items():
    for logic_gate,gate_inits in logic_gates.items():
      node_name=tech_map[logic_gate]
      port=gates_vlib_2[node_name]['port']
      for init_name in gate_inits:
        tmp={}
        for mIO,mNode in zip(gates_vlib_2[node_name]["inputs"],gate_inits[init_name]['inputs']):
          tmp[mIO]=mNode
        tmp[gates_vlib_2[node_name]["outputs"][0]]=gate_inits[init_name]['outputs']
        txt+=f"{node_name} {init_name} {port.format(**tmp)}\n"
  return txt

In [ ]:
def FF_to_txt_techmap(FF_tech,FF_vlib_2,tech_map):
  # tech_map={target:[source]}
  txt=""
  for FF_tech_i in FF_tech:
    node_name=tech_map[FF_tech_i]
    port=FF_vlib_2[node_name]['port']
    for init_name in FF_tech[FF_tech_i]:
      tmp={}
      for mIO in FF_vlib_2[node_name]["port_list"]:
        tmp[mIO]=FF_tech[FF_tech_i][init_name][utils.det_FF_node(mIO)]
      txt+= f"{node_name} {init_name} {port.format(**tmp)}\n"
  return txt

In [ ]:
FF

In [ ]:
txt=gates_to_txt_techmap(locked.top_module.gates,gates_vlib2,tech_map)
txt+=FF_to_txt_techmap(locked.top_module.FF_tech,FF_vlib2,tech_map)

In [ ]:
with open("tmp.v","w") as f:
    f.write(txt)

In [ ]:
# path_dir=r"/mnt/d/alis_files/LAPTOP/alis_files/university_files/GDRIVE/Hardware security FYP/gnnre/Netlist_to_graph/Circuits_datasets/Interconnected-Modules"
# paths=[os.path.join(path_dir,i) for i in os.listdir(path_dir) if(i[:5]=="Train")]

# import re
# def analyze_init(paths):
#   items={}
#   for path in paths:
#     with open(path,"r") as f:
#       module_txt=f.read()
    
#     for i in re.findall(r"(\w+) (\w+) \((.*)\);",module_txt)[1:]:
#       tmpi=re.findall(r'\.(\S+)\( ?[^\(\),]+\)',i[2])
#       port="("
#       for k in tmpi[:-1]:
#         port+=f".{k}({{{k}}}), "
#       port+=f".{tmpi[-1]}({{{tmpi[-1]}}})"+");"
#       if(i[0] not in items):
#         items[i[0]]=[port]
#       elif(items[i[0]]!=[port]):
#         raise Exception("Items have mismatching initiation")
#   return items

In [ ]:
# items=analyze_init(paths)

In [ ]:

# with open("input_files/ASSURE_LOCKED/modulefiles.v","r") as f:
#   module_txt=f.read()


# with open("tmp/tmpmod.v","w") as f:
#   f.write(module_txt)


# # with open("tmp.v","w") as f:
# #   f.write(t)

# gate_mapping,gates,FF=verilog_parser.extract_modules_def(module_txt)

# path="input_files/ASSURE_LOCKED/design4/design4_netlist.v"
# # "input_files/test.v"
# # "input_files/ASSURE_LOCKED/design4/design4_netlist.v"




# # # verilog=tmp1()
# # verilog=tmp2()
# # with open("tmp/tmp.v","w") as f:
# #   f.write(verilog)
# # # print(verilog)

# # print("HERE")


In [ ]:
# gate_tech,sub_module,(FF_tech,Clock_pins,Reset_pins)=verilog_parser.gates_module_extraction(verilog,gate_mapping,gates,FF)